In [1]:
%matplotlib inline
import random
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import matplotlib.cm as cm
from scipy.stats import norm
import scipy.sparse as sparse
import time
import datetime
import seaborn as sns
sns.set()
plt.rcParams['font.family'] = 'IPAexGothic'

In [2]:
#============================
# 京都_国道171号ルート_初期設定
#============================

df_root_001 = pd.read_csv('./CSV/004_京都_国道171号ルート/004_京都_国道171号ルート_初期設定.csv')
df_root_001 = df_root_001.sort_values(by = '距離') 
df_root_001

,市区町村コード,居住地,目的地,距離,選択ルート備考,鉄道定期券利用者数
0,272051,吹田市,吹田市役所,8.5,国道171号ルート,35287
1,272116,茨木市,茨木市役所,17.0,国道171号ルート,19968
2,272078,高槻市,高槻市役所,23.7,国道171号ルート,22528
3,273015,島本町,島本町役場,30.8,国道171号ルート,2956
4,263036,大山崎町,大山崎町役場,33.5,国道171号ルート,139
5,262099,長岡京市,長岡京市役所,37.7,国道171号ルート,3024
6,262081,向日市,向日市役所,40.9,国道171号ルート,1326
7,261009,京都市,京都市役所,50.2,国道171号ルート,22673
8,252018,大津市,大津市役所,61.5,国道171号ルート,5479


In [3]:
#============================
# 徒歩帰宅者年齢グループ_初期設定
#============================

df_group_setting = pd.read_csv('./CSV/徒歩帰宅者グループ設定/group_setting.csv')

# 連続徒歩行動可能時間のリスト
allotted_time_list = df_group_setting['allotted_time']
# 次の帰宅行動のために取る休憩時間のリスト
recess_Time_list = df_group_setting['recess_Time']
# 疲労係数のリスト
fatigue_coefficient_list = df_group_setting['fatigue_coefficient']
# 徒歩帰宅の速度のリスト
pace_walking_list = df_group_setting['pace_walking']
df_group_setting

,Group,allotted_time,recess_Time,fatigue_coefficient,pace_walking
0,A,8,16,0.8,2.2
1,B,10,14,0.9,2.6
2,C,12,12,1.0,2.8


In [4]:
#######################################
# 初期化
#######################################

# 通過自治体のリスト
place_list = ['出発地']
place_list.extend(df_root_001['居住地'].unique())

# 通過自治体の数
place_number = len(place_list)

# 通過自治体の距離リスト
disrance_list = df_root_001['距離']

#　出発地点の自治体のリスト
start_list = ['大阪市']
start_list.extend(df_root_001['居住地'].unique())

# 対象地の徒歩帰宅者全員がリタイア者となった場合に累積リタイア者数の計算を無効化するためのフラグ
retire_flag_list = [0] * len(place_list)

In [5]:
disrance_list

0     8.5
1    17.0
2    23.7
3    30.8
4    33.5
5    37.7
6    40.9
7    50.2
8    61.5
Name: 距離, dtype: float64

In [6]:
#######################################
# ルートの初期設定の保存
#######################################
df_root_setting = df_root_001.copy()
df_root_setting.to_csv('./Result/model_set_A_004_京都_国道171号ルート/model_set_A_004_ルート_初期設定.csv', index = False)
df_root_setting

# 目的地ごとの距離を保存するリスト
distance_list = [0.0]
distance_series = df_root_setting['距離']
for i in range(len(distance_series)) :
    # kmをmに単位変換
    dis = float(distance_series[i] * 1000)
    distance_list.append(dis)
    #print(dis)


In [7]:
#######################################
# クラスの定義
#######################################

"""
Walking_Model_Case_1

・徒歩帰宅行動者の徒歩時速は、一律に pace_walking (m/min) とする。
・徒歩帰宅行動者の連続徒歩行動可能時間の初期値は、allotted_time (min)とする。
・連続徒歩行動可能時間を使った後、recess_Time 分の休憩をとることで、再び徒歩帰宅行動が可能となるが、
次の連続徒歩行動可能時間はそれまでの行動の疲労蓄積を考慮して、前回に行動した時間に係数　fatigue_coefficient　を乗じた値に減少していくものとする。

Group,allotted_time,recess_Time,fatigue_coefficient,pace_walking
A,8,16,0.8,2.2
B,10,14,0.9,2.6
C,12,12,1,2.8

"""
# シミュレーション計算を行う日数
day_num = 7

#　GroupB
# 連続徒歩行動可能時間の初期値 (分)
allotted_time = 12 * 60
# 次の帰宅行動のために取る休憩時間（分）
recess_Time = 12 * 60
# 疲労係数（前回の徒歩行動時間との積を次の徒歩行動可能時間とする）
fatigue_coefficient = 1.0
# 徒歩帰宅の速度（m/min）
pace_walking = 2.8 * 1000 / 60



In [8]:
#######################################
# クラスの定義
#######################################

class Walking_Model_Case_1:
    
    def __init__(self, allotted_time, recess_Time, fatigue_coefficient, pace_walking, goal):
        # 連続徒歩行動可能時間の初期値
        self.default_allotted_Time = allotted_time
        # 次の帰宅行動のために取る休憩時間の初期値
        self.my_recess_Time = recess_Time
        # 疲労係数の初期値
        self.my_fatigue_coefficient = fatigue_coefficient
        # 徒歩帰宅の速度の初期値
        self.my_pace_walking = pace_walking
        # 目的地までの距離
        self.my_goal_distance = goal

        # 現在の徒歩行動の行動可能時間
        self.current_allotted_Time = self.default_allotted_Time
        # 徒歩行動の行動可能時間の変化を保存するリスト
        self.allotted_Time_list = []
        # 現在の徒歩行動の累積時間
        self.current_walking_Time = 0
        # 現在の徒歩行動の累積距離
        self.current_Distance = 0
        # すべての徒歩行動を合計した時間
        self.total_walking_Time = 0
        # すべての徒歩行動を合計した距離
        self.total_Distance = 0
        
        # 徒歩帰宅行動の回数
        self.walk_count = 0
        # 休憩行動の回数
        self.recess_count = 0
        
        # 現在の休憩の時間
        self.current_recess_Time = 0
        # すべての休憩を合計した時間
        self.total_recess_Time = 0
        
        # すべての行動時間（休憩も含む）
        self.total_time = 0
        
        # 状態
        # 0 は出発前
        # 1 は徒歩行動
        # 2 は休憩
        # 3 は帰宅行動終了
        # 4 はリタイア（行動不可）
        self.status = 0
        
    def get_status_String(self):
        status_str = ''
        if self.status == 0:
            status_str = '出発前モード'
        elif self.status == 1:
            status_str = '徒歩行動モード'
        elif self.status == 2:
            status_str = '休憩モード'
        elif self.status == 3:
            status_str = '帰宅行動終了モード'
        elif self.status == 4:
            status_str = 'リタイア（行動不可）モード'
            
        result_str = ''
        remaining = 0
        if self.total_Distance < self.my_goal_distance:
            result_str = '帰宅行動未完了'
            remaining = self.my_goal_distance - self.total_Distance
        else:
            result_str = '帰宅行動完了'
            
        #print('-'*33, ' {}'.format(status_str))
        #print('-'*33, ' {} : 残りの距離 {} km'.format(result_str, round(remaining / 1000, 2)))
        
    def gcurrent_allotted_Time(self):
        n = self.walk_count - 1
        self.current_allotted_Time = round(self.default_allotted_Time * (self.my_fatigue_coefficient ** n), 3)
        self.allotted_Time_list.append(self.current_allotted_Time)
        
        #print('self.walk_count : ', self.walk_count)
        #print('n : ', n)
        #print('my_fatigue_coefficient ** n : ', self.my_fatigue_coefficient ** n)    
        #print('self.current_allotted_Time : ',self.current_allotted_Time)
        #return self.current_allotted_Time
    
        
    # 1分ずつシミュレーションを実行する
    def walk(self, min):
        # 目的地までの距離が0の場合
        if self.my_goal_distance == 0:
            return
        
        # 帰宅行動終了モード or リタイア（行動不可）モードの場合は何もしない
        if self.status == 3 or self.status == 4:
            return
        
        #print('min =  {} / status =  {}'.format(min, self.status))
        # 初期状態
        if self.status == 0:
            # 出発
            #print('-'*33, '初期モードから徒歩行動モードに移行')
            self.status = 1
            
            # カウンターを 0 スタート
            self.walk_count += 1
            self.gcurrent_allotted_Time()
            
        # 徒歩行動
        elif self.status == 1:
            #print('current_walking_Time =  {} / current_allotted_Time =  {}'.format(self.current_walking_Time, self.current_allotted_Time))
            #print('total_Distance =  {} / my_goal_distance =  {}'.format(self.total_Distance, self.my_goal_distance))
            # 現在の徒歩行動の累積時間が、現在の徒歩行動の行動可能時間を超えていない
            if self.current_walking_Time <= self.current_allotted_Time:
                # 徒歩行動
                self.current_walking_Time += 1
                self.total_walking_Time += 1
                self.total_time += 1
                self.current_Distance += self.my_pace_walking
                self.total_Distance += self.my_pace_walking
                
                # 距離チェック
                if self.my_goal_distance <= self.total_Distance:
                    
                    # 目的地に到着
                    #print('-'*33, '徒歩行動モードから到着モードに移行')
                    self.status = 3
                    # 戻す
                    self.current_walking_Time = 0
                    self.current_Distance = 0
                
            else:
                # 行動可能時間を超えたので、
                # 徒歩行動を終了し、休憩行動に切り替える
                #print('-'*33, '徒歩行動モードから休憩モードに移行')
                self.status = 2
                # 戻す
                self.current_walking_Time = 0
                self.current_Distance = 0
                self.recess_count += 1
            
        # 休憩
        elif self.status == 2:
            #print('current_recess_Time =  {} / total_recess_Time =  {}'.format(self.current_recess_Time, self.total_recess_Time))
            if self.current_recess_Time < self.my_recess_Time:
                # 現在の休憩の累積時間が、休憩最大時間を超えていない
                self.current_recess_Time += 1
                self.total_recess_Time += 1
                self.total_time += 1
            else:
                #print('current_recess_Time =  {} / my_recess_Time =  {}'.format(self.current_recess_Time, self.my_recess_Time))
                # 休憩最大時間を超えたので、休憩を終了し、徒歩行動に切り替える
                #print('-'*33, '休憩モードから徒歩行動モードに移行')
                self.status = 1
                
                self.walk_count += 1
                self.current_recess_Time = 0
                self.gcurrent_allotted_Time()
            

In [9]:
#######################################
# シミュレーション結果を保存するdataframeの作成
#######################################

# 通過自治体の数
dataframe_colmns =[
    "目的地",
    "目的地までの距離 (km)",
    "移動距離 (km)",
    "連続徒歩行動可能時間の初期値 (時)",
    "徒歩時間 (時)",
    "休憩時間 (時)",
    "すべての時間 (時)",
    "帰宅行動回数 (回)",
    "休憩回数 (回)"
                  ]

# "連続徒歩行動可能時間の履歴 (時)"
# 目的地ごとの到着者数を保存するデータフレームdf_001の作成
rows = place_number
cols = len(dataframe_colmns)
npdata = np.zeros((rows, cols))
df_zero = pd.DataFrame(npdata) # 行の名前を設定
df_zero.columns = dataframe_colmns
df_zero = df_zero.astype('float64')
# string型のカラムを追加
df_zero['連続徒歩行動可能時間の履歴 (分)'] = '-'
df_zero['帰宅行動回数 (回)'] = df_zero['帰宅行動回数 (回)'].astype('int64')
df_zero['休憩回数 (回)'] = df_zero['休憩回数 (回)'].astype('int64')
df_001 = df_zero.copy()

# 目的地を表すカラムを追加
df_001['目的地'] = start_list
# 目的地までの距離を追加
df_001['目的地までの距離 (km)'] = distance_list
#df_001

In [10]:
for i in range(len(distance_list)):
    # 対象となる距離(m)を取得
    kyori = int(distance_list[i])
    
    """
    # 連続徒歩行動可能時間の初期値 (分)
allotted_time = 6 * 60

# 次の帰宅行動のために取る休憩時間（分）
recess_Time = 18 * 60

# 疲労係数（前回の徒歩行動時間との積を次の徒歩行動可能時間とする）
fatigue_coefficient = 0.75

# 徒歩帰宅の速度（m/min）
pace_walking = 3 * 1000 / 60
    """
    
    # モデルのインスタンス作成
    instance = Walking_Model_Case_1(allotted_time, 
                                    recess_Time, 
                                    fatigue_coefficient, 
                                    pace_walking, 
                                    kyori)

    # 行動シミュレーション（時間単位ごとにシミュレーションを実行する）
    for j in range(day_num * 24 * 60):
        instance.walk(j)
        
    # dataframe にシミュレーション結果を出力
    for k in range(len(df_001.columns)):
        if k == 1:
            # 目的地までの距離 (km)
            df_001.iat[i, k] = round(distance_list[i] / 1000, 2)
        elif k == 2:
            # 移動距離 (km)
            df_001.iat[i, k] = round(instance.total_Distance / 1000, 2)
        elif k == 3:
            # 連続徒歩行動可能時間の初期値 (時)
            df_001.iat[i, k] = round(instance.default_allotted_Time / 60, 0)
        elif k == 4:
            # 徒歩時間 (時)
            df_001.iat[i, k] = round(instance.total_walking_Time / 60, 2)
        elif k == 5:
            # 休憩時間 (時)
            df_001.iat[i, k] = round(instance.total_recess_Time / 60, 2)
        elif k == 6:
            # すべての時間 (時)
            df_001.iat[i, k] = round(instance.total_time / 60, 2)
        elif k == 7:
            # 帰宅行動回数 (回)
            df_001.iat[i, k] = instance.walk_count
        elif k == 8:
            # 休憩回数 (回)
            df_001.iat[i, k] = instance.recess_count
        elif k == 9:
            # 連続徒歩行動可能時間の履歴 (分)
            df_001.iat[i, k] = instance.allotted_Time_list

# 結果をCSVに出力
df_001.to_csv('./Result/model_set_A_004_京都_国道171号ルート/model_set_A_004_GroupC_目的地ごとのシミュレーション結果.csv', index = False)

In [11]:
df_001

,目的地,目的地までの距離 (km),移動距離 (km),連続徒歩行動可能時間の初期値 (時),徒歩時間 (時),休憩時間 (時),すべての時間 (時),帰宅行動回数 (回),休憩回数 (回),連続徒歩行動可能時間の履歴 (分)
0,大阪市,0.0,0.00,12.0,0.00,0.0,0.00,0,0,[]
1,吹田市,8.5,8.54,12.0,3.05,0.0,3.05,1,0,[720.0]
2,茨木市,17.0,17.03,12.0,6.08,0.0,6.08,1,0,[720.0]
3,高槻市,23.7,23.71,12.0,8.47,0.0,8.47,1,0,[720.0]
4,島本町,30.8,30.80,12.0,11.00,0.0,11.00,1,0,[720.0]
5,大山崎町,33.5,33.51,12.0,11.97,0.0,11.97,1,0,[720.0]
6,長岡京市,37.7,37.71,12.0,13.47,12.0,25.47,2,1,"[720.0, 720.0]"
7,向日市,40.9,40.93,12.0,14.62,12.0,26.62,2,1,"[720.0, 720.0]"
8,京都市,50.2,50.21,12.0,17.93,12.0,29.93,2,1,"[720.0, 720.0]"
9,大津市,61.5,61.51,12.0,21.97,12.0,33.97,2,1,"[720.0, 720.0]"


In [12]:
# 最大帰宅行動回数の取得
max_walk_count = max(df_001['帰宅行動回数 (回)'])
max_walk_count

# 移動と休憩のdataframe
df_002_colmns =[
    "目的地"
                  ]
for i in range(max_walk_count * 2) :
    txt = '';
    s = 0
    if i % 2 == 1:
        s = (i + 1) / 2
        txt = '休憩'
    else:
        s = (i + 2) / 2
        txt = '移動'

    str = '{}{}'.format(txt, int(s))
    df_002_colmns.append(str)

rows = place_number
cols = len(df_002_colmns)
repetitions = int((cols - 1)/2)
npdata = np.zeros((rows, cols))
df_002 = pd.DataFrame(npdata)
df_002.columns = df_002_colmns
# 目的地を表すカラムを追加
df_002['目的地'] = start_list
df_002

,目的地,移動1,休憩1,移動2,休憩2
0,大阪市,0.0,0.0,0.0,0.0
1,吹田市,0.0,0.0,0.0,0.0
2,茨木市,0.0,0.0,0.0,0.0
3,高槻市,0.0,0.0,0.0,0.0
4,島本町,0.0,0.0,0.0,0.0
5,大山崎町,0.0,0.0,0.0,0.0
6,長岡京市,0.0,0.0,0.0,0.0
7,向日市,0.0,0.0,0.0,0.0
8,京都市,0.0,0.0,0.0,0.0
9,大津市,0.0,0.0,0.0,0.0


In [13]:
for i in range(rows):
    #print(i)
    # df_001から連続徒歩行動可能時間の履歴 (分)のリストを取得
    listA = df_001.iat[i, 9]
    listAnum = len(listA) * 2
    for j in range(cols):
        if i > 0 and j > 0:
            if j <= listAnum:
                if j % 2 == 1:
                    # 移動時間
                    timeNum = 0
                    if j == (listAnum - 1):
                        # 最後の徒歩行動時間
                        # 行動可能時間の合計から、最後の要素を減算した timeA を取得
                        timeA = sum(listA) - listA[-1]
                        # df_001から合計徒歩行動時間 timeB を取得
                        timeB = df_001.iat[i, 4] * 60
                        # 最後の徒歩行動時間 timeC を計算
                        timeC = timeB - timeA
                        timeNum = round(timeC / 60, 2)
                    else:
                        # 最後ではない途中の徒歩行動時間（連続徒歩行動可能時間と同じ）
                        c = int(j / 2)
                        timeNum = round(listA[c] / 60, 2)
                        
                    df_002.iat[i, j] = timeNum
                else:
                    # 休憩時間
                    if j != listAnum:
                        df_002.iat[i, j] = round(recess_Time / 60, 2)
                
# 結果をCSVに出力
df_002.to_csv('./Result/model_set_A_004_京都_国道171号ルート/model_set_A_004_GroupC_目的地ごとの移動と休憩時間の結果.csv', index = False)

In [14]:
df_002

,目的地,移動1,休憩1,移動2,休憩2
0,大阪市,0.00,0.0,0.00,0.0
1,吹田市,3.05,0.0,0.00,0.0
2,茨木市,6.08,0.0,0.00,0.0
3,高槻市,8.47,0.0,0.00,0.0
4,島本町,11.00,0.0,0.00,0.0
5,大山崎町,11.97,0.0,0.00,0.0
6,長岡京市,12.00,12.0,1.47,0.0
7,向日市,12.00,12.0,2.62,0.0
8,京都市,12.00,12.0,5.93,0.0
9,大津市,12.00,12.0,9.97,0.0
